In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb

# 1. CSV 파일 로드
df = pd.read_csv("./dataset/healthcare_sampled.csv")  # your_file.csv 파일 경로 입력
categories = df["category"].unique()
diseases = df["disease_name"].unique()
print(categories)
print(len(diseases))

/root/anaconda3/envs/llm/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


['귀코목질환' '뇌신경정신질환' '소아청소년질환' '소화기질환' '여성질환' '유방내분비질환' '치과질환' '호흡기질환']
248


In [3]:
import random
# 2. SentenceTransformer로 임베딩 모델 불러오기
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 3. Chroma DB 클라이언트 초기화
client = chromadb.PersistentClient("./chroma_db_storage")

categories_en = [
    "ENT",              # 귀코목질환
    "Neuropsych",       # 뇌신경정신질환
    "Pediatrics",       # 소아청소년질환
    "Gastrointestinal", # 소화기질환
    "Gynecology",       # 여성질환
    "Breast/Endocrine", # 유방내분비질환
    "Dental",           # 치과질환
    "Respiratory"       # 호흡기질환
]

# 4. Category별로 response를 벡터스토어에 저장
for i in range(len(categories_en)):
    category = categories[i]
    category_en = categories_en[i]
    # 각 카테리별 response 텍스트를 무작위로 최대 30개 샘플링
    category_responses = df[df["category"] == category]["response"].tolist()
    category_responses = random.sample(category_responses, min(30, len(category_responses)))
    print(category, len(category_responses))

    # response 텍스트를 임베딩
    embeddings = embedding_model.encode(category_responses)
    
    # Chroma DB 컬렉션 생성 및 데이터 추가
    collection_name = f"{category_en}_chroma_db"
    collection = client.create_collection(name=collection_name)

    # 각 문서에 고유한 ID 할당
    ids = [f"{category}_{i}" for i in range(len(category_responses))]

    # 벡터 추가
    collection.add(
        documents=category_responses,
        embeddings=embeddings,
        metadatas=[{"category": category}] * len(category_responses),
        ids=ids  # 고유한 ID 리스트 추가
    )

    print(f"'{collection_name}' 컬렉션에 {len(category_responses)}개의 문서가 저장되었습니다.")




ENT 0


UniqueConstraintError: Collection ENT_chroma_db already exists

In [ ]:

from sentence_transformers import SentenceTransformer
import chromadb
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# 1. 모델 및 벡터스토어 초기화
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # 벡터 임베딩 모델
client = chromadb.PersistentClient()

# gemma 모델 로드
tokenizer = AutoTokenizer.from_pretrained("finetuned-gemma-model")  # finetuned-gemma-model 경로로 변경
gemma_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned-gemma-model")

# 2. LSTM으로 category 예측 (예시용 함수)
def predict_category(question):
    # 여기에 미리 학습된 LSTM 모델로 category 예측 코드 추가
    # 예시로 반환된 category
    return "predicted_category"

# 3. 질문을 받아 RAG 수행하는 함수
def generate_answer_with_rag(question):
    # (1) LSTM으로 category 예측
    category = predict_category(question)
    
    # (2) 예측된 category에 맞는 벡터스토어 로드
    collection_name = f"{category}_chroma_db"
    collection = client.get_collection(name=collection_name)
    
    # (3) 질문을 벡터로 임베딩하고 유사한 response 검색
    question_embedding = embedding_model.encode(question)
    results = collection.query(query_embeddings=[question_embedding], n_results=5)
    
    # 검색 결과에서 response 텍스트 가져오기
    context_texts = [result["document"] for result in results["documents"][0]]
    context = " ".join(context_texts)  # 검색된 response를 하나의 문맥으로 결합
    
    # (4) gemma 모델로 답변 생성 (RAG 수행)
    inputs = tokenizer(question + " " + context, return_tensors="pt", truncation=True, max_length=256)
    with torch.no_grad():
        outputs = gemma_model.generate(**inputs, max_length=256, num_return_sequences=1)
    
    # 생성된 답변 디코딩
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return answer

# 예시 질문
question = "How does the process work for my category?"
answer = generate_answer_with_rag(question)
print("Generated Answer:", answer)